## Environment

Please excute the following instruction in command line before started

 `! pip install Backtesting`

In [1]:
pip install Backtesting

Note: you may need to restart the kernel to use updated packages.


In [2]:
! pip install pandas

# Package modification issue


Modify line 148 in

`/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py`

to

`new_bar_idx = new_index.get_indexer([mean_time], method='nearest')[0]`

In [3]:
import fileinput
import sys

# (some bug in backtesting package need to be fixed)
# file path( change it to your file path) 
file_path = "XXXXXXXPython310\\Lib\\site-packages\\backtesting\\_plotting.py"
# Lines to be modified
line_to_edit = 148

# New content
new_content = "                new_bar_idx = new_index.get_indexer([mean_time], method='nearest')[0]"

# Security Check: Backup original file
backup_file_path = file_path + '.backup'
with open(file_path, 'r') as original_file, open(backup_file_path, 'w') as backup_file:
    backup_file.write(original_file.read())

# Attempt to modify the file
try:
    with fileinput.FileInput(file_path, inplace=True, backup='.bak') as file:
        for i, line in enumerate(file, start=1):
            if i == line_to_edit:
                print(new_content)  # Replace with new content
            else:
                print(line, end='')  # Keep the original content

    print("File modification successful.")
except Exception as e:
    print(f"Error occurred: {e}", file=sys.stderr)
    # Restore original file in case of an error
    with open(backup_file_path, 'r') as backup_file, open(file_path, 'w') as original_file:
        original_file.write(backup_file.read())
    print("File has been restored to its original state.", file=sys.stderr)


File modification successful.


# Main Program

### Dowanload Ta-Lib 

- colab download Ta-lib



In [62]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

- Windows download TA-Lib

find in https://www.lfd.uci.edu/~gohlke/pythonlibs/#ta-lib

then download it and remember the path

go to the path and open cmd

for example：`cd C:\Users\user\Downloads`

input ` pip install \[ .whl file full names]`
for example： `pip install TA_Lib‑0.4.24‑cp310‑cp310‑win_amd64.whl`


### For example:

In [1]:
cd "C:\Users\userXXXXXXXXXXXX"

[WinError 2] 系統找不到指定的檔案。: 'C:\\Users\\userXXXXXXXXXXXX'
c:\Users\user\OneDrive - 國立陽明交通大學\Python\final_project


C:\Users\user\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [5]:
pip install TA_Lib-0.4.24-cp310-cp310-win_amd64.whl

Processing c:\users\user\onedrive - 國立陽明交通大學\temp\ta_lib-0.4.24-cp310-cp310-win_amd64.whl
TA-Lib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


## Import Data

In [6]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
from backtesting.test import SMA, GOOG
import pandas as pd
import talib

# Please replace it with the path to your own file on the cloud(or the local file path).
# Please download the EURUSD_M1_train.csv file first which in the folder 
#data_path = "Your csv file path"  
data_path = "EURUSD_M1_train.csv"

#Import Data.
df = pd.read_csv(data_path, names=GOOG.columns)
df.index = pd.to_datetime(df.index)
df.drop(index=df.index[0], inplace=True)
df = df.astype("float")

df
df.head()

#Each transaction format.
'''
,Open,High,Low,Close,Volume
20170102 020000,1.05155,1.05197,1.05155,1.0519,0
20170102 020100,1.05209,1.05209,1.05177,1.05179,0
20170102 020200,1.05177,1.05198,1.05177,1.05178,0
'''

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\Users\user\AppData\Local\Temp\ipykernel_28316\2011744394.py:13: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import 

'\n,Open,High,Low,Close,Volume\n20170102 020000,1.05155,1.05197,1.05155,1.0519,0\n20170102 020100,1.05209,1.05209,1.05177,1.05179,0\n20170102 020200,1.05177,1.05198,1.05177,1.05178,0\n'

## Strategy

In [7]:
class SmaCross(Strategy):
    # Define the time window lengths for two moving averages
    n1 = 885  # Longer-term moving average period
    n2 = 244  # Shorter-term moving average period
    # Best n1 = 885, n2 = 244, size = 10000000 0.461163
    def init(self):
        # Initialize the strategy
        close = self.data.Close  # Get closing price data
        self.sma1 = self.I(SMA, close, self.n1)  # Calculate the longer-term moving average
        self.sma2 = self.I(SMA, close, self.n2)  # Calculate the shorter-term moving average

    def next(self):
        # Define the execution logic of the strategy
        # Golden cross: A bullish signal where the shorter-term moving average crosses above the longer-term moving average
        if crossover(self.sma1, self.sma2):
            # Execute a buy if the shorter-term moving average crosses above the longer-term moving average
            self.buy(size=10000000)
        elif crossover(self.sma2, self.sma1):
            # Execute a sell if the shorter-term moving average crosses below the longer-term moving average
            # Only long positions, no short positions
            if len(self.trades) > 0:
                self.trades[0].close()

'''
Golden Cross represents a signal of an upward trend when a faster-moving average or indicator crosses above a slower-moving one.
Death Cross represents a signal of a downward trend when a faster-moving average or indicator crosses below a slower-moving one.
'''

class TripleCross(Strategy):
    # Define the time window lengths for three moving averages
    n1 = 548  # Period for the first moving average
    n2 = 206  # Period for the second moving average
    n3 = 118  # Period for the third moving average
    # Best n1 = 548, n2 = 206, n3 = 118 return 67.2% sharp 1.07
    # Ratio approximately 55:21:13
    def init(self):
        # Initialize the strategy
        close = self.data.Close  # Get closing price data
        self.sma1 = self.I(SMA, close, self.n1)  # Calculate the first moving average
        self.sma2 = self.I(SMA, close, self.n2)  # Calculate the second moving average
        self.sma3 = self.I(SMA, close, self.n3)  # Calculate the third moving average

    def next(self):
        count = 0
        if crossover(self.sma1, self.sma3) and crossover(self.sma2, self.sma3):
            # Execute a buy signal if the shorter-term and middle moving averages cross above the longer-term moving average
            self.buy()
        elif crossover(self.sma3, self.sma2) and crossover(self.sma3, self.sma1):
            # Execute a sell signal if the shorter-term and middle moving averages cross below the longer-term moving average
            self.sell()

class RSICross(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_window = 133
    # Best upper_bound 70 lower_bound 30, rsi 133 return 47 sharp ratio 0.81

    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)

    def next(self):
        if crossover(self.rsi, self.upper_bound):
            # Execute a buy if the RSI crosses above the upper bound
            self.buy()
        elif crossover(self.rsi, self.lower_bound):
            # Execute a sell if the RSI crosses below the lower bound
            self.sell()

## Backtesting

### Set up
* Initial Amount：100000000
* Transaction Fee：0.00005

In [8]:
bt = Backtest(df, RSICross,cash=100000000, commission = 0.00005, exclusive_orders=True)

output = bt.run()
output

C:\Users\user\AppData\Local\Temp\ipykernel_28316\4080375678.py:1: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, RSICross,cash=100000000, commission = 0.00005, exclusive_orders=True)


Start                     2017-01-02 02:00:00
End                       2022-12-30 16:58:00
Duration                   2188 days 14:58:00
Exposure Time [%]                   94.868862
Equity Final [$]             151697064.548124
Equity Peak [$]              152015967.269874
Return [%]                          51.697065
Buy & Hold Return [%]                1.772032
Return (Ann.) [%]                    5.763621
Volatility (Ann.) [%]                6.563643
Sharpe Ratio                         0.878113
Sortino Ratio                        1.410603
Calmar Ratio                         0.700257
Max. Drawdown [%]                   -8.230726
Avg. Drawdown [%]                   -0.145551
Max. Drawdown Duration      531 days 08:21:00
Avg. Drawdown Duration        1 days 11:45:00
# Trades                                   25
Win Rate [%]                             76.0
Best Trade [%]                       9.822729
Worst Trade [%]                     -2.309871
Avg. Trade [%]                    

### Create Analysis Chart

- If you use colab, please skip the following instruction and just run the bt.plot()

In [17]:
 pip install --force-reinstall bokeh==2.4.3

     ---------------------------------------- 0.0/18.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/18.5 MB 1.3 MB/s eta 0:00:15
     ---------------------------------------- 0.1/18.5 MB 1.1 MB/s eta 0:00:18
     --------------------------------------- 0.1/18.5 MB 901.1 kB/s eta 0:00:21
     --------------------------------------- 0.2/18.5 MB 952.6 kB/s eta 0:00:20
     ---------------------------------------- 0.2/18.5 MB 1.1 MB/s eta 0:00:18
      --------------------------------------- 0.3/18.5 MB 1.1 MB/s eta 0:00:17
      --------------------------------------- 0.3/18.5 MB 1.0 MB/s eta 0:00:18
      --------------------------------------- 0.4/18.5 MB 1.1 MB/s eta 0:00:18
      --------------------------------------- 0.4/18.5 MB 1.0 MB/s eta 0:00:18
     - -------------------------------------- 0.5/18.5 MB 1.1 MB/s eta 0:00:17
     - -------------------------------------- 0.5/18.5 MB 1.1 MB/s eta 0:00:17
     - -------------------------------------- 0.6/18.5 MB

If you use Vscode or other editor please open the html with

"Brave" browser or "Microsoft Edge" browser

In [18]:
bt.plot()


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2497', ...)